<details>
   <summary>Metadata</summary>
    title: "Introduction to the Google Earth Engine Python API: Sentinel-2 cloud masking, interactive mapping and download"<br>
    description: "This is a tutorial within the second theme of Module 1 of the E-TRAINEE course."<br>
    lastUpdate: 2023-07-14<br>
    author: Andreas Mayr<br>
</details>

# Introduction to the Google Earth Engine Python API: Sentinel-2 cloud masking, interactive mapping and download

This Notebook performs a procedure for cloud masking in Sentinel-2 imagery, which is an important basis for further time series analysis. It is shown how to (i) access [Google Earth Engine](https://developers.google.com/earth-engine/) (GEE) image collections via the GEE Python API, (ii) process the imagery in the cloud, (iii) visualize the results in interactive maps, and (iv) download data from GEE. More specifically, you will learn how to achieve the following steps:

- Query and filter GEE image collections of Sentinel-2 imagery and associated cloud probabilities
- Join the two different, filtered image collections to build a new collection
- Derive clouds and cloud shadows as components of a cloud mask
- Dilate the edge of cloud mask objects
- Visualize the imagery, the final cloud mask and its components in an interactive Leaflet map using the [Folium](https://python-visualization.github.io/folium/) package
- Download cloud masked data produced in the Earth Engine to a local hard drive (using the [geemap](https://github.com/giswqs/geemap#installation) package)
- Apply a function (e.g. for calculating a vegetation index) to an image collection

Much of the Notebook is based on [Sentinel-2 Cloud Masking with s2cloudless](https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless) by [Justin Braaten](https://github.com/jdbcode), licensed under the [CC BY 4.0 License](https://creativecommons.org/licenses/by/4.0/) and the [Apache 2.0 License](https://www.apache.org/licenses/LICENSE-2.0) (code samples).

*Requirements:*

* You will need a Google account with GEE activated ([sign up here](https://earthengine.google.com/signup/), if you do not already have one).
* The Python packages used in this tutorial are listed in the requirements file provided for the course. Please see the instructions on the [software page](https://3dgeo-heidelberg.github.io/etrainee/software/software_python.html) for setting up a Conda environment based on this file.

## Authenticate and initialize API

First, we load the required packages: Google Earth Engine Python API (which is called `ee`), `geemap` and `folium`.

In [ ]:
import ee
import geemap
import folium

Before we can start using the GEE API, we must authenticate with our credentials and initialize the API. Run the cell and follow the instructions on how to grant the notebook access with your account.

In [ ]:
try:                            # If you have authenticated recently, you might be able to initialize directly
        ee.Initialize()
except Exception as e:          # If initialize does not work, you probably have to authenticate first
        ee.Authenticate()
        ee.Initialize()


Successfully saved authorization token.


## Query and filter image collections

We want to query the Sentinel-2 Multi-Spectral instrument (MSI) image collection with Level-2A (surface reflectance) products and apply filters to the collection. First, we define a geometry and some other parameters that will be used for filtering:

In [ ]:
my_point = ee.Geometry.Point(11.0, 46.8)    # Point of interest with latitude and longitude
AOI = my_point                              # Area of interest (map display will be centered on this, here we simply use a point)
START_DATE = '2021-06-09'                   # Start date of our time period of interest
END_DATE = '2021-07-11'                     # End date of our time period of interest
CLOUD_FILTER = 60                           # Maximum image cloud cover percent allowed in image collection
CLD_PRB_THRESH = 40                         # Cloud probability (%); values greater than are considered cloud
NIR_DRK_THRESH = 0.15                       # NIR reflectance; values less than this are considered potential cloud shadow
CLD_PRJ_DIST = 1.5                          # Maximum distance (km) to search for cloud shadows from cloud edges
BUFFER = 60                                 # Distance [m] to dilate the edge of cloud-identified objects

You may fine tune the cloud-mask related thresholds to get the best results for your needs. The strictness of cloud masking is essentially a trade-off between data completeness (little spatial and temporal gaps) and reliability of land surface observations. It will depend on your further processing steps (such as compositing) and how any remaining cloudy pixels can compromise the reliability of intermediate products and final results.

We will use two different ImageCollections:
- The [Sentinel-2 Level-2A collection](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR) provides the surface reflectances produced by the sen2cor processor and downloaded from the Copernicus Open Access Hub.
- The [Sentinel 2 Cloud Probability collection](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_CLOUD_PROBABILITY) is created using a machine learning approach to cloud detection, which is detailed [here](https://medium.com/sentinel-hub/improving-cloud-detection-with-machine-learning-c09dc5d7cf13) and [here](https://docs.sentinel-hub.com/api/latest/user-guides/cloud-masks/#cloud-masks-and-cloud-probabilities). Alternatively, the QA60 cloud mask or the SCL scene classification map provided by [ESA](https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-2-msi/level-2a/algorithm) with the Sentinel-2 L2A product could be used. See [this blog post](https://medium.com/google-earth/more-accurate-and-flexible-cloud-masking-for-sentinel-2-images-766897a9ba5f) for a comparative discussion of the cloud masks.

These two collections are filtered by bounds and date and then joined into a single collection.

In [5]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 surface reflectance
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter cloud probabilities produced by s2cloudless
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered cloud probabilities to the surface reflectances by the 'system:index' property
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

# Apply the function defined above to build a collection containing both S2 SR and cloud probabilities 
my_S2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)


## Define functions for cloud mask components

### Cloud components
Define a function to add the s2cloudless probability layer and derived cloud mask as bands to an S2 SR image input.

In [6]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value (defined at the beginning).
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))


## Cloud shadow components

Define a function to add dark pixels, cloud projection, and identified shadows as bands to an S2 SR image input. Note that the image input needs to be the result of the above add_cloud_bands function because it relies on knowing which pixels are considered cloudy ('clouds' band).

In [7]:
def add_shadow_bands(img):
    # Exclude water pixels using the scene classification (SCL) band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

## Final cloud-shadow mask
Define a function to assemble all of the cloud and cloud shadow components and produce the final mask.

In [8]:
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)
    # or (to include only the final cloud/cloud shadow mask along with the original image bands)
    #return img.addBands(is_cld_shdw)

## Visualize and evaluate cloud mask components

### Define functions to display image and mask component layers.

Folium will be used to display map layers. Import folium and define a method to display Earth Engine image tiles.

In [9]:
# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

Define a function to display all of the cloud and cloud shadow components to an interactive Folium map. The input is an image collection where each image is the result of the add_cld_shdw_mask function defined previously.

To facilitate display in a single map, we create a mosaic of the collection. [ee.ImageCollection.mosaic](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-mosaic) composites images according to their position in the collection (priority is last to first) and pixel mask status, where invalid (mask value 0) pixels are filled by preceding valid (mask value >0) [pixels.

In [10]:
def display_cloud_layers(col):
    # Mosaic the image collection.
    img = col.mosaic()

    # Subset layers and prepare them for display.
    clouds = img.select('clouds').selfMask()
    shadows = img.select('shadows').selfMask()
    dark_pixels = img.select('dark_pixels').selfMask()
    probability = img.select('probability')
    cloudmask = img.select('cloudmask').selfMask()
    cloud_transform = img.select('cloud_transform')

    # Create a folium map object.
    center = AOI.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=12)

    # Add layers to the folium map.
    m.add_ee_layer(img,
                   {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                   'S2 image', True, 1, 9)
    m.add_ee_layer(probability,
                   {'min': 0, 'max': 100},
                   'probability (cloud)', False, 1, 9)
    m.add_ee_layer(clouds,
                   {'palette': 'e056fd'},
                   'clouds', False, 1, 9)
    m.add_ee_layer(cloud_transform,
                   {'min': 0, 'max': 1, 'palette': ['white', 'black']},
                   'cloud_transform', False, 1, 9)
    m.add_ee_layer(dark_pixels,
                   {'palette': 'orange'},
                   'dark_pixels', False, 1, 9)
    m.add_ee_layer(shadows, {'palette': 'yellow'},
                   'shadows', False, 1, 9)
    m.add_ee_layer(cloudmask, {'palette': 'orange'},
                   'cloudmask', True, 0.5, 9)

    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)

    # Export the map as HTML file (to be viewed in a web browser), this is optional.
    outfp = "output/cloud_mask_map.html"
    m.save(outfp)

### Display mask component layers

Map the add_cld_shdw_mask function over the collection to add mask component bands to each image, then display the results.

Give the system some time to render everything, it should take less than a minute.

In [11]:
my_S2_sr_cld_col_disp = my_S2_sr_cld_col.map(add_cld_shdw_mask)
display_cloud_layers(my_S2_sr_cld_col_disp)

### Apply cloud mask

We use these cloud mask components to set the cloud-affected pixels to NoData.

In [12]:
# Define a function to apply the cloud mask to each image in the collection
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

# Add the mask components to each image and apply the function to process the collection
S2_sr_cloudless = my_S2_sr_cld_col.map(add_cld_shdw_mask).map(apply_cld_shdw_mask)

### Build a cloud-free composite

Reduce the collection by median (in your application, you might consider using medoid reduction to build a composite from actual data values, instead of per-band statistics).
For comparison, we also create a mosaic (filled with most recent pixel values, see above). Moreover, we want to know how many cloud-free observations are available for each pixel and, thus, use the `count()` funtion.

In [13]:
s2_sr_median = S2_sr_cloudless.median()
s2_sr_mosaic = S2_sr_cloudless.mosaic()
s2_sr_count = S2_sr_cloudless.count()

### Display the cloud-free composites

Display the results. Be patient while the map renders, it may take a minute; `ee.Image.reproject()` is forcing computations to happen at 100 and 20 m scales (i.e. it is not relying on appropriate pyramid level scales for analysis). The issues with `ee.Image.reproject()` being resource-intensive in this case are mostly confined to interactive map viewing. Batch image exports and table reduction exports where the scale parameter is set to typical Sentinel-2 scales (10-60 m) are less affected. For the count layer's colour scale a simple legend would be good but we skip this for simplicity here ([this tutorial](https://www.youtube.com/watch?v=-rO1MztlLMo) describes adding legends with the geemap package).

In [14]:
# Create a folium map object.
center = AOI.centroid(10).coordinates().reverse().getInfo()
m = folium.Map(location=center, zoom_start=12)

# Add layers to the folium map.
m.add_ee_layer(s2_sr_count,
                {'bands': ['B4'], 'min': 0, 'max': 20},
                'S2 cloud-free count', True, 1, 9)
m.add_ee_layer(s2_sr_median,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 cloud-free median', True, 1, 9)
m.add_ee_layer(s2_sr_mosaic,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 cloud-free mosaic', True, 1, 9)

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)


## Export images to local machine

### Export the composite image

First, we will export the composite (i.e. a single image) for our study area to a folder on our local hard drive. Here, we make use of [geemap](https://geemap.org/geemap), a Python package described in [Wu (2020)](https://doi.org/10.21105/joss.02305) The geemap tools facilitate interactive mapping but also some other tasks with Google Earth Engine greatly.

We define a region for the export as a polygon.
Clipping seems not to be necessary then. Nevertheless, see [here](https://stackoverflow.com/questions/61220828/how-to-clip-an-image-collection-to-a-feature-collections-geometry) for clipping image collections, or [here](https://stackoverflow.com/questions/70589018/how-to-download-image-collection-from-google-earth-engine) for how to use filterBounds.

In [16]:
# Define a region-of-interest (ROI) from a list of GeoJSON 'Polygon' formatted coordinates (Lon-Lat pairs).
geom = ee.Geometry.Polygon(
        [ 
            [ 
                [10.990786901656424, 46.845543799137999],
                [10.990786901656424, 46.878736564096471],
                [11.055145371135509, 46.878736564096471],
                [11.055145371135509, 46.845543799137999],
                [10.990786901656424, 46.845543799137999]    # matching the first vertex is optional
            ]
        ]
    )
feature = ee.Feature(geom, {})
my_roi = feature.geometry()

Now we define an output path and actually export ...
Using either geemap or [this solution](https://courses.spatialthoughts.com/end-to-end-gee.html#batch-exports). Note that we export all 12 bands into a single GeoTIFF file.

In [17]:
ee_object = s2_sr_median
file_name = "C:\work\etrainee\gee\s2_sr_median.tif"
geemap.ee_export_image(ee_object, file_name, scale=10, region=my_roi, file_per_band=False)  # Scale sets the resolution in meters per pixel. Defaults to 1000.

Generating URL ...
Please wait ...
Data downloaded to C:\work\etrainee\gee\s2_sr_median.tif


### Export the cloud-masked image collection

We use geemap because this contains a nice function to export data directly to a local hard drive.

In [18]:
ee_object = S2_sr_cloudless
out_dir = "C:\work\etrainee\gee"
geemap.ee_export_image_collection(ee_object, out_dir, scale=10, region=my_roi, file_per_band=False)
print("Finished!")

Total number of images: 9

Exporting 1/9: 20210610T101559_20210610T102220_T32TPS.tif
Generating URL ...
Please wait ...
Data downloaded to C:\work\etrainee\gee\20210610T101559_20210610T102220_T32TPS.tif


Exporting 2/9: 20210612T101031_20210612T101329_T32TPS.tif
Generating URL ...
Please wait ...
Data downloaded to C:\work\etrainee\gee\20210612T101031_20210612T101329_T32TPS.tif


Exporting 3/9: 20210615T102021_20210615T102602_T32TPS.tif
Generating URL ...
Please wait ...
Data downloaded to C:\work\etrainee\gee\20210615T102021_20210615T102602_T32TPS.tif


Exporting 4/9: 20210617T100559_20210617T101301_T32TPS.tif
Generating URL ...
Please wait ...
Data downloaded to C:\work\etrainee\gee\20210617T100559_20210617T101301_T32TPS.tif


Exporting 5/9: 20210622T101031_20210622T101416_T32TPS.tif
Generating URL ...
Please wait ...
Data downloaded to C:\work\etrainee\gee\20210622T101031_20210622T101416_T32TPS.tif


Exporting 6/9: 20210625T102021_20210625T102545_T32TPS.tif
Generating URL ...
Please

### Calculate the NDVI from the cloud-masked image collection and export

From the collection of cloudmasked S2 images we calculate the NDVI. In GEE there is a helper function `normalizedDifference()` for such purposes and a more flexible `expression()` function. We use `lambda` to define a function and then [map it over the collection](https://developers.google.com/earth-engine/guides/ic_mapping). Using `map()` over an image collection is similar to a for-loop but it can make the computations running in parallel.
In addition, we print the number of scenes in this collection.

Finally, we will export the cloud-free NDVI image collection for the ROI to a directory on our local hard drive. Again, we make use of [geemap](https://geemap.org/geemap).

In [19]:
S2_ndvi = S2_sr_cloudless.map(lambda image: image.normalizedDifference(['B8', 'B4']).rename(['ndvi']))
print(S2_ndvi.__sizeof__())
ee_object = S2_ndvi
out_dir = "C:\work\etrainee\gee"
geemap.ee_export_image_collection(ee_object, out_dir, scale=10, region=my_roi, file_per_band=False)
print("Finished!")

32
Finished!
